### Imports

In [96]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps.storymap import JournalStoryMap
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer, Feature
from arcgis.geometry import Geometry, SpatialReference, Point

### Utility Methods

In [120]:
import uuid,json
def getExtent(fromFeature):
    g = Geometry(fromFeature.geometry)
    if g.type.lower() == 'point':
        bg = Point({"x": g.x, "y": g.y, "spatialReference": {"wkid": 102100}})
        return bg.extent
    
    return g.extent

def getMapAction(fromFeature, forWebmapId, label_field="OBJECTID", label_prefix="Feature Extent", add_popup=False):
    # TODO - make sure input parameters are valid, since we are just inserting them into the mapaction schema
    
    # pull out a value from the feature to display in our anchor tag
    lbl_val = fromFeature.attributes[fromFeature.fields[0]]
    if label_field in fromFeature.fields:
        lbl_val = fromFeature.attributes[label_field]
    
    # format the label
    lbl_frmtd = '{0}: {1}'.format(label_prefix, lbl_val)
    
    # get the geometry extent
    # TODO - Need to validate this or default to a valid extent
    
    ext = getExtent(fromFeature)
    
    # create a unique id
    # ts = int(round(time.time() * 1000)) this isn't precise enough, gets duplicated in loop
    ts = str(uuid.uuid4())
    #print('id : {0}'.format(ts))
    
    # insert values into our map action schema
    ma = {
        "id":"MJ-ACTION-{0}".format(ts),
        "type": "media",
        "media": {
            "type":"webmap",
            "webmap": {
                "id":forWebmapId,
                "extent": {
                    "xmin": ext[0],
                    "ymin": ext[1],
                    "xmax": ext[2],
                    "ymax": ext[3],
                    "spatialReference":{
                        "wkid": 102100
                    }
                },
                "layers": None,
                "popup": None,
                    # "layerId": "hosted_point_layer_8467",
                    #  "fieldName": "OBJECTID",
                    #  "fieldValue": 6,
                    #  "anchorPoint": {
                    #       "x": -13063833.789716866,
                    #       "y": 4013607.1718674107,
                    #        "spatialReference": {
                    #           "wkid": 102100
                    #       }
                    #   }
                "overview":{
                    "enable": False,
                    "openByDefault": False
                },
                "legend": {
                    "enable": False,
                    "openByDefault": False
                },
                "geocoder":{
                    "enable": False
                },
                "altText": ""
            }
        }
    }
    
    # create the html string
    content_anchor = '<p><a data-storymaps="{0}" data-storymaps-type="media">{1}</a></p><p>&nbsp;</p>'.format(ma['id'], lbl_frmtd)
    
    # hand back our results to whoever called this function
    return {"content_link": content_anchor, "map_action": ma}

### Connect to Portal

In [3]:
prtl = GIS(username='marktorrey')

Enter password: ········


### Get a webmap that has some data we want to query

In [57]:
wm_itm = prtl.content.get('1ad33784a4824ef38f2ce1c60b2963e0')
wm = WebMap(wm_itm)
point_layer = FeatureLayer(wm.layers[0].url)
poly_layer = FeatureLayer(wm.layers[2].url)

### Get the story map

In [58]:
itm = prtl.content.get('876f3dc003184f2ea1d2785e294fd983')
sm = JournalStoryMap(itm)
sections = sm.properties['values']['story']['sections']

### Clear out our test section

In [117]:
# clear out any stuff we tried previously
sections[1]['content'] = 'No Content Defined' 
sections[1]['contentActions'] = []
sm.save()

True

### Iterate over our features and generate the map actions

In [118]:
poly_features = poly_layer.query()
point_features = point_layer.query()
for f in point_features.features:
    ma = getMapAction(f, wm_itm['id'], label_field='textfield')
    sections[1]['content'] += ma['content_link']
    sections[1]['contentActions'].append(ma['map_action'])
    
sm.save()


True